In [1]:
import sys
!{sys.executable} -m pip install wandb optuna "ray[tune]" -q

In [2]:
import os
os.chdir('/workspace')

In [3]:
!cd /workspace && git clone https://github.com/Eran-BA/PoT.git

fatal: destination path 'PoT' already exists and is not an empty directory.


In [ ]:
import wandb
import os

# Set the NEW key
os.environ["WANDB_API_KEY"] = ""

# Force re-login
wandb.login(key=os.environ["WANDB_API_KEY"], relogin=True, force=True)

# Test it
run = wandb.init(project="sudoku-hpo-test", name="test-run")
wandb.log({"test": 1})
wandb.finish()
print("✓ Success!")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eranbt92 (eranbt92-open-university-of-israel) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


test,▁
test,1


✓ Success!


In [5]:

import os
import sys
import glob
import torch
import wandb
from datetime import datetime
# Download artifact
import wandb
run = wandb.init()
artifact = run.use_artifact('eranbt92-open-university-of-israel/sudoku-finetune/sudoku-finetune-best:v8', type='model')
artifact_dir = artifact.download()
wandb.finish()

# Find the checkpoint file
checkpoint_path = os.path.join(artifact_dir, "trial_lr5.95e-04_L4_asyncTrue_best.pt")
if not os.path.exists(checkpoint_path):
    # List files to find the right one
    print("Files in artifact:")
    for f in os.listdir(artifact_dir):
        print(f"  {f}")
    checkpoint_path = os.path.join(artifact_dir, os.listdir(artifact_dir)[0])

# Load checkpoint
checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)

print(f"✓ Loaded checkpoint:")
print(f"  Epoch: {checkpoint['epoch']}")
print(f"  Best Grid Acc: {checkpoint['best_grid_acc']:.2f}%")
print(f"  Config: {checkpoint['config']}")

wandb: Downloading large artifact 'sudoku-finetune-best:v8', 271.07MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:00.3 (875.6MB/s)
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Files in artifact:
  best_model.pt
✓ Loaded checkpoint:
  Epoch: 1450
  Best Grid Acc: 42.10%
  Config: {'lr': 0.000594874681321977, 'weight_decay': 0.2624967163228772, 'puzzle_lr_multiplier': 32.72290672827732, 'puzzle_weight_decay': 1.4293571265737601, 'H_cycles': 2, 'L_cycles': 4, 'halt_max_steps': 2, 'halt_exploration': 0.09275410183585496, 'dropout': 0.007625738023228556, 'beta2': 0.9106812512723371, 'warmup_steps': 2500, 'async_batch': True, 'batch_size': 768, 'epochs_per_trial': 2000, 'eval_interval': 100, 'checkpoint_interval': 100, 'checkpoint_dir': '/workspace/PoT/experiments/hpo_results/checkpoints', 'd_model': 512, 'n_heads': 8, 'H_layers': 2, 'L_layers': 2, 'd_ff': 2048, 'T': 4, 'hrm_grad_style': True, 'wandb_project': 'sudoku-hpo', 'wandb_group': 'sudoku_hpo_4xB200'}


In [6]:
os.chdir('/workspace/PoT/')

In [7]:
from src.data import download_sudoku_dataset

# Download dataset
download_sudoku_dataset(
    output_dir='data/sudoku-extreme-10k-aug-100',
    subsample_size=10000,
    num_aug=100,
)

Note: Augmentation is now ON-THE-FLY (not pre-computed)
  Train puzzles: 9000, Val puzzles: 1000


Processing train: 100%|██████████| 9000/9000 [00:00<00:00, 73291.83it/s]


  train: 9000 puzzles (augmentation: on-the-fly)


Processing val: 100%|██████████| 1000/1000 [00:00<00:00, 72276.96it/s]


  val: 1000 puzzles (augmentation: on-the-fly)


Processing test: 422786it [00:05, 71228.40it/s]


  test: 422786 puzzles
✓ Dataset saved to data/sudoku-extreme-10k-aug-100


In [ ]:

# ============================================================
# PHASE 2 LR + HALT_STEPS Search
# ============================================================
import os
import sys
import torch
import ray
from ray import tune
from datetime import datetime

os.chdir('/workspace/PoT')
sys.path.insert(0, '/workspace/PoT')


# Config from checkpoint (already loaded)
config = checkpoint['config']
# Initialize Ray
ray.init(num_gpus=1, ignore_reinit_error=True)

# Load checkpoint
checkpoint_ref = ray.put(checkpoint)

def train_trial(trial_config):
    import os
    import sys
    import math
    import torch
    import wandb
    from torch.utils.data import DataLoader
    from ray.air import session
    
    sys.path.insert(0, '/workspace/PoT')
    os.chdir('/workspace/PoT')
    
    from src.data import SudokuDataset
    from src.pot.models import HybridPoHHRMSolver
    from src.training import train_epoch_async, evaluate, log_halt_histogram_to_wandb
    
    import ray
    checkpoint = ray.get(trial_config['checkpoint_ref'])
    config = checkpoint['config']
    
    halt_max_steps = trial_config['halt_max_steps']
    lr = trial_config['lr']
    
    device = torch.device('cuda')
    
    # Initialize W&B for this trial
    run = wandb.init(
        project="sudoku-finetune",
        name=f"halt{halt_max_steps}_lr{lr:.0e}",
        config={'halt_max_steps': halt_max_steps, 'lr': lr},
        reinit=True,
    )
    print(f"✓ W&B: {run.url}")

        # Append to URLs file
    with open('/workspace/wandb_run_urls.txt', 'a') as f:
        f.write(f"{run.url} | halt={halt_max_steps}, lr={lr:.2e}\n")
    # Data
    train_dataset = SudokuDataset('data/sudoku-extreme-10k-aug-100', 'train')
    val_dataset = SudokuDataset('data/sudoku-extreme-10k-aug-100', 'val')
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)
    
    # Model
    model = HybridPoHHRMSolver(
        d_model=config['d_model'], n_heads=config['n_heads'],
        H_layers=config['H_layers'], L_layers=config['L_layers'],
        d_ff=config['d_ff'], dropout=config['dropout'],
        H_cycles=config['H_cycles'], L_cycles=config['L_cycles'],
        T=config['T'], num_puzzles=1, hrm_grad_style=config['hrm_grad_style'],
        halt_max_steps=halt_max_steps,
        halt_exploration_prob=config['halt_exploration'],
    ).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Optimizer
    puzzle_lr = lr * config['puzzle_lr_multiplier']
    puzzle_params = list(model.puzzle_emb.parameters())
    model_params = [p for p in model.parameters() if p not in set(puzzle_params)]
    
    optimizer = torch.optim.AdamW(model_params, lr=lr, weight_decay=config['weight_decay'], betas=(0.9, config['beta2']))
    puzzle_optimizer = torch.optim.AdamW(puzzle_params, lr=puzzle_lr, weight_decay=config['puzzle_weight_decay'], betas=(0.9, config['beta2']))
    
    epochs = 500
    total_steps = epochs * len(train_loader)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=lr * 0.1)
    puzzle_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(puzzle_optimizer, T_max=total_steps, eta_min=puzzle_lr * 0.1)
    
    best_grid_acc = checkpoint['best_grid_acc']
    save_dir = '/tmp/phase2_checkpoints'
    os.makedirs(save_dir, exist_ok=True)
    best_path = os.path.join(save_dir, f'best_halt{halt_max_steps}_lr{lr:.0e}.pt')
    
    for epoch in range(1, epochs + 1):
        # Train with halt histogram on eval epochs
        train_metrics = train_epoch_async(
            model, train_loader, optimizer, puzzle_optimizer,
            device, epoch, use_poh=True,
            scheduler=scheduler, puzzle_scheduler=puzzle_scheduler,
            track_halt_histogram=(epoch % 50 == 0),  # Track on eval epochs
        )
        train_dataset.on_epoch_end()
        
        if epoch % 50 == 0 or epoch == 1:
            # Evaluate with halt histogram
            val_metrics = evaluate(model, val_loader, device, use_poh=True, track_halt_histogram=True)
            
            # Prepare W&B log dict
            log_dict = {
                'epoch': epoch,
                'train_loss': train_metrics['loss'],
                'train_cell_acc': train_metrics['cell_acc'],
                'train_grid_acc': train_metrics['grid_acc'],
                'val_loss': val_metrics['loss'],
                'val_grid_acc': val_metrics['grid_acc'],
                'val_cell_acc': val_metrics['cell_acc'],
                'best_grid_acc': best_grid_acc,
            }
            
            # Log TRAIN halt histogram
            if 'halt_histogram' in train_metrics:
                train_halt_log = log_halt_histogram_to_wandb(train_metrics['halt_histogram'], prefix="train_")
                log_dict.update(train_halt_log)
                if "train_halt_steps_raw" in train_halt_log and train_halt_log["train_halt_steps_raw"]:
                    log_dict["train_halt_histogram"] = wandb.Histogram(train_halt_log["train_halt_steps_raw"])
            
            # Log VAL halt histogram
            if 'halt_histogram' in val_metrics:
                val_halt_log = log_halt_histogram_to_wandb(val_metrics['halt_histogram'], prefix="val_")
                log_dict.update(val_halt_log)
                if "val_halt_steps_raw" in val_halt_log and val_halt_log["val_halt_steps_raw"]:
                    log_dict["val_halt_histogram"] = wandb.Histogram(val_halt_log["val_halt_steps_raw"])
            
            wandb.log(log_dict)
            
            if val_metrics['grid_acc'] > best_grid_acc:
                best_grid_acc = val_metrics['grid_acc']
                
                # Save locally
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'halt_max_steps': halt_max_steps,
                    'lr': lr,
                    'best_grid_acc': best_grid_acc,
                    'config': {**config, 'halt_max_steps': halt_max_steps},
                }, best_path)
                
                # Upload to W&B (overwrites with "best" alias)
                artifact = wandb.Artifact(f"phase2-halt{halt_max_steps}", type="model")
                artifact.add_file(best_path)
                run.log_artifact(artifact, aliases=["best", "latest"])
                print(f"  🏆 New best: {best_grid_acc:.2f}% → uploaded to W&B")
            
            session.report({
                'train_loss': train_metrics['loss'],
                'val_grid_acc': val_metrics['grid_acc'],
                'best_grid_acc': best_grid_acc,
                'epoch': epoch,
            })
            print(f"[halt={halt_max_steps}, LR={lr:.2e}] Ep {epoch}: grid={val_metrics['grid_acc']:.1f}%, best={best_grid_acc:.1f}%")
    
    wandb.finish()




# Define experiments: (halt_max_steps, lr_multiplier)
original_lr = config['lr']  # 5.95e-4

experiments = [
    # halt_max_steps=3, 3 LRs
    {'halt_max_steps': 3, 'lr': original_lr * 0.001},  # 5.95e-7
    {'halt_max_steps': 3, 'lr': original_lr * 0.01},   # 5.95e-6
    {'halt_max_steps': 3, 'lr': original_lr * 0.05},   # 2.97e-5
    # halt_max_steps=4, 2 medium LRs
    {'halt_max_steps': 4, 'lr': original_lr * 0.005},  # 2.97e-6
    {'halt_max_steps': 4, 'lr': original_lr * 0.01},   # 5.95e-6
]

print("Experiments to run:")
for i, exp in enumerate(experiments):
    print(f"  {i+1}. halt_max_steps={exp['halt_max_steps']}, lr={exp['lr']:.2e}")

# Run all experiments
tuner = tune.Tuner(
    tune.with_resources(train_trial, resources={'gpu': 1, 'cpu': 4}),
    param_space={
        'halt_max_steps': tune.grid_search([e['halt_max_steps'] for e in experiments]),
        'lr': tune.grid_search([e['lr'] for e in experiments]),
        'checkpoint_ref': checkpoint_ref,
    },
    tune_config=tune.TuneConfig(
        num_samples=1,
        max_concurrent_trials=1,
    ),
    run_config=ray.air.RunConfig(
        name='phase2_lr_halt_search',
        stop={'epoch': 500},
    ),
)

# Actually need to pass combinations properly - use grid_search with list of dicts
tuner = tune.Tuner(
    tune.with_resources(train_trial, resources={'gpu': 1, 'cpu': 4}),
    param_space=tune.grid_search([
        {'halt_max_steps': 3, 'lr': original_lr * 0.001, 'checkpoint_ref': checkpoint_ref},
        {'halt_max_steps': 3, 'lr': original_lr * 0.01, 'checkpoint_ref': checkpoint_ref},
        {'halt_max_steps': 3, 'lr': original_lr * 0.05, 'checkpoint_ref': checkpoint_ref},
        {'halt_max_steps': 4, 'lr': original_lr * 0.01, 'checkpoint_ref': checkpoint_ref},
    ]),
    tune_config=tune.TuneConfig(
        num_samples=1,
        max_concurrent_trials=1,
    ),
    run_config=ray.air.RunConfig(
        name='phase2_lr_halt_search',
        stop={'epoch': 500},
    ),
)

results = tuner.fit()

# Print results
print("\n" + "="*60)
print("SEARCH RESULTS")
print("="*60)
for result in results:
    halt = result.config['halt_max_steps']
    lr = result.config['lr']
    acc = result.metrics.get('best_grid_acc', 0)
    print(f"halt_max_steps={halt}, LR={lr:.2e}: {acc:.2f}%")

best = results.get_best_result(metric='best_grid_acc', mode='max')
print(f"\n🏆 BEST: halt_max_steps={best.config['halt_max_steps']}, LR={best.config['lr']:.2e}")
print(f"   Grid Acc: {best.metrics['best_grid_acc']:.2f}%")
'''

**5 experiments total:**
1. halt=3, lr=5.95e-7 (0.001x)
2. halt=3, lr=5.95e-6 (0.01x)
3. halt=3, lr=2.97e-5 (0.05x)
4. halt=4, lr=5.95e-6 (0.01x)

~7.5 hours total on 1x H200.
'''

(train_trial pid=1203366) wandb: Currently logged in as: eranbt92 (eranbt92-open-university-of-israel) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(train_trial pid=1203366) wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


(train_trial pid=1203366) ✓ W&B: https://wandb.ai/eranbt92-open-university-of-israel/sudoku-finetune/runs/6cmss1p2
(train_trial pid=1203366) [train] Loaded 9000 puzzles
(train_trial pid=1203366)   Augmentation: ON-THE-FLY
(train_trial pid=1203366) [val] Loaded 1000 puzzles
(train_trial pid=1203366)   Augmentation: OFF


(train_trial pid=1203366) wandb: Tracking run with wandb version 0.23.1
(train_trial pid=1203366) wandb: Run data is saved locally in /workspace/PoT/wandb/run-20251208_170720-6cmss1p2
(train_trial pid=1203366) wandb: Run `wandb offline` to turn off syncing.
(train_trial pid=1203366) wandb: Syncing run halt3_lr6e-07
(train_trial pid=1203366) wandb: ⭐️ View project at https://wandb.ai/eranbt92-open-university-of-israel/sudoku-finetune
(train_trial pid=1203366) wandb: 🚀 View run at https://wandb.ai/eranbt92-open-university-of-israel/sudoku-finetune/runs/6cmss1p2
Epoch 1 (async):  96%|█████████▌| 8841/9216 [00:22<00:00, 409.72it/s, loss=0.5501, cell_acc=77.3%, grid_acc=21.5%]
(pid=gcs_server) [2025-12-08 17:07:44,829 E 1201823 1201823] (gcs_server) gcs_server.cc:303: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
Epoch 1 (async)

(train_trial pid=1203366) 
(train_trial pid=1203366) 📊 Halt Step Histogram:
(train_trial pid=1203366) Step   Halted     Solved     Solve Rate  
(train_trial pid=1203366) ----------------------------------------
(train_trial pid=1203366) 3      1000       574        57.40     %
(train_trial pid=1203366) 


Evaluating: 100%|██████████| 2/2 [00:01<00:00,  1.58it/s]


(train_trial pid=1203366)   🏆 New best: 57.40% → uploaded to W&B
(train_trial pid=1203366) [halt=3, LR=5.95e-07] Ep 1: grid=57.4%, best=57.4%


(train_trial pid=1203366) /venv/main/lib/python3.12/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_trial pid=1203366)   _log_deprecation_warning(
Epoch 2 (async):   8%|▊         | 779/9216 [00:02<00:19, 436.28it/s, loss=0.5844, cell_acc=77.2%, grid_acc=19.3%]
(pid=1202060) [2025-12-08 17:07:46,036 E 1202060 1203159] core_worker_process.cc:837: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14 [repeated 18x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.ht

In [ ]:

# ============================================================
# Final: Re-train best config and evaluate on test set
# ============================================================
print("\n" + "="*60)
print("Training best config for final evaluation...")
print("="*60)


best_halt = best.config['halt_max_steps']
best_lr = best.config['lr']

# Rebuild model with best halt_max_steps
model = HybridPoHHRMSolver(
    d_model=config['d_model'], n_heads=config['n_heads'],
    H_layers=config['H_layers'], L_layers=config['L_layers'],
    d_ff=config['d_ff'], dropout=config['dropout'],
    H_cycles=config['H_cycles'], L_cycles=config['L_cycles'],
    T=config['T'], num_puzzles=1, hrm_grad_style=config['hrm_grad_style'],
    halt_max_steps=best_halt,
    halt_exploration_prob=config['halt_exploration'],
).to(device)

# Load BEST TRAINED weights (not original checkpoint!)
best_trial_path = f'/tmp/phase2_checkpoints/best_halt{best_halt}_lr{best_lr:.0e}.pt'
if os.path.exists(best_trial_path):
    best_ckpt = torch.load(best_trial_path, map_location=device, weights_only=False)
    model.load_state_dict(best_ckpt['model_state_dict'])
    print(f"✓ Loaded best trained model: {best_ckpt['best_grid_acc']:.2f}%")
else:
    print(f"⚠️ Best model not found at {best_trial_path}, using original checkpoint")
    model.load_state_dict(checkpoint['model_state_dict'])
# Test evaluation
test_dataset = SudokuDataset('data/sudoku-extreme-10k-aug-100', 'test')
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)

test_metrics = evaluate(model, test_loader, device, use_poh=True)

print(f"\n🎯 TEST RESULTS (422k puzzles):")
print(f"   Cell Accuracy: {test_metrics['cell_acc']:.2f}%")
print(f"   Grid Accuracy: {test_metrics['grid_acc']:.2f}%")

# Upload to W&B
run = wandb.init(
    project="sudoku-finetune",
    name=f"best_halt{best_halt}_lr{best_lr:.0e}",
    config={
        'halt_max_steps': best_halt,
        'lr': best_lr,
        'best_grid_acc': best.metrics['best_grid_acc'],
    }
)

wandb.log({
    'test_cell_acc': test_metrics['cell_acc'],
    'test_grid_acc': test_metrics['grid_acc'],
    'val_best_grid_acc': best.metrics['best_grid_acc'],
})

# Save and upload best model
save_dir = 'experiments/hpo_results/phase2_best'
os.makedirs(save_dir, exist_ok=True)
best_path = os.path.join(save_dir, f'best_halt{best_halt}_model.pt')

torch.save({
    'model_state_dict': model.state_dict(),
    'halt_max_steps': best_halt,
    'lr': best_lr,
    'test_grid_acc': test_metrics['grid_acc'],
    'config': {**config, 'halt_max_steps': best_halt},
}, best_path)

artifact = wandb.Artifact(
    name="sudoku-phase2-best",
    type="model",
    description=f"Best Phase 2 model - halt={best_halt}, lr={best_lr:.2e}, test_grid={test_metrics['grid_acc']:.2f}%"
)
artifact.add_file(best_path)
wandb.log_artifact(artifact)

wandb.finish()
print(f"\n✓ Done! Best model saved to: {best_path}")